<a href="https://colab.research.google.com/github/elenaporras-mx/Project7_AI/blob/main/project7_demo1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AlphaZero-Style MCTS: Play Tic-Tac-Toe vs. the Agent
This notebook lets you play as 'O' against an AlphaZero-style MCTS agent playing 'X'.
- Agent uses a fake neural net (priors + value) to guide tree search.
- You choose moves using input().
- Board updates every turn until the game ends.


- **What to Watch:** We’ll visualize how the performance of the agent improves over training iterations. Early on, it plays randomly. After a few iterations of self-play and training, you’ll see the agent start to prefer stronger moves (as identified by the combination of its network and search). We can plot the win-rate of the agent against an earlier version of itself over training time – you should see it climbing.
- **Outcome:** By the end of the demo training, the agent should play the game at a competent level (possibly perfectly if the game is simple like Tic-Tac-Toe). This will mirror, in miniature, the process AlphaZero underwent – though of course our demo is far smaller in scale.
- **Key Point:** Even in this small example, using MCTS to assist learning can dramatically accelerate the learning of the network, compared to learning the game solely by policy gradient or value iteration without lookahead.

In [ ]:
# === game & mcts definitions ===
import numpy as np
import matplotlib.pyplot as plt

class TicTacToe:
   def __init__(self):
       self.board = [' '] * 9  # create empty 3x3 board
       self.current_player = 'X'  # x goes first

   def legal_moves(self):
       # return indexes of empty spaces
       return [i for i in range(9) if self.board[i] == ' ']

   def make_move(self, idx):
       # try to place mark at position idx
       if self.board[idx] == ' ':
           self.board[idx] = self.current_player
           self.current_player = 'O' if self.current_player == 'X' else 'X'  # swap players
           return True
       return False

   def check_winner(self):
       # all possible winning combinations
       wins = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
       for a,b,c in wins:
           if self.board[a] != ' ' and self.board[a] == self.board[b] == self.board[c]:
               return self.board[a]  # return winning player
       if ' ' not in self.board:
           return 'Draw'
       return None

   def print_board(self):
       # display board with separators
       for i in range(0, 9, 3):
           print(' | '.join(self.board[i:i+3]))
           if i < 6:
               print('--+---+--')

In [ ]:
# === alphazero-style mcts ===
class AZNode:
    def __init__(self, state, parent=None, move=None, prior=1.0):
        self.state = state
        self.parent = parent
        self.move = move
        self.prior = prior  # probability from policy network
        self.children = []
        self.visits = 0
        self.wins = 0

    def uct_score(self, c=1.41):
        # calculate exploration score with prior probability
        if self.visits == 0:
            return self.prior + 1e-8  # prevent division by zero
        return (self.wins / self.visits) + c * self.prior * np.sqrt(np.log(self.parent.visits + 1) / self.visits)

In [ ]:
def dummy_nn_prior_value(board):
    # fake neural network giving uniform priors over legal moves
    priors = {i: 1/9 for i in range(9) if board[i] == ' '}
    return priors, 0.0

In [ ]:
def simulate_random_game(start):
    # play out a random game from given state
    game = TicTacToe()
    game.board = start.board[:]
    game.current_player = start.current_player
    while game.check_winner() is None:
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
    return game.check_winner()

In [ ]:
def az_mcts_search(root_state, sims=100):
    # conduct monte carlo tree search with alphazero improvements
    priors, _ = dummy_nn_prior_value(root_state.board)
    root = AZNode(root_state, prior=1.0)
    # initialize tree with all possible moves
    for move, prior in priors.items():
        g = TicTacToe()
        g.board = root_state.board[:]
        g.current_player = root_state.current_player
        g.make_move(move)
        root.children.append(AZNode(g, parent=root, move=move, prior=prior))

    # run simulations
    for _ in range(sims):
        node = root
        path = [node]
        # selection phase - traverse tree using uct
        while node.children:
            node = max(node.children, key=lambda n: n.uct_score())
            path.append(node)
        # simulation and backpropagation
        result = simulate_random_game(node.state)
        for n in path:
            n.visits += 1
            if result == root_state.current_player:
                n.wins += 1
            elif result == 'Draw':
                n.wins += 0.5
    return root

In [ ]:
# === gameplay loop ===
game = TicTacToe()
while game.check_winner() is None:
    if game.current_player == 'X':
        # ai makes a move
        root = az_mcts_search(game, sims=100)
        best = max(root.children, key=lambda c: c.visits)  # pick most visited move
        game = best.state
        print("\nAgent (X) moves:")
        game.print_board()
    else:
        # human player's turn
        game.print_board()
        print("Your move (O). Choose one of these:", game.legal_moves())
        move = int(input("Enter your move (0-8): "))
        # validate input
        while move not in game.legal_moves():
            move = int(input("Invalid. Try again (0-8): "))
        game.make_move(move)
        print("You moved.")

# game over
winner = game.check_winner()
game.print_board()
print("\nGame Over! Winner:", winner)


Agent (X) moves:
X |   |  
--+---+--
  |   |  
--+---+--
  |   |  
X |   |  
--+---+--
  |   |  
--+---+--
  |   |  
Your move (O). Choose one of these: [1, 2, 3, 4, 5, 6, 7, 8]
Enter your move (0-8): 4
You moved.

Agent (X) moves:
X | X |  
--+---+--
  | O |  
--+---+--
  |   |  
X | X |  
--+---+--
  | O |  
--+---+--
  |   |  
Your move (O). Choose one of these: [2, 3, 5, 6, 7, 8]
Enter your move (0-8): 2
You moved.

Agent (X) moves:
X | X | O
--+---+--
X | O |  
--+---+--
  |   |  
X | X | O
--+---+--
X | O |  
--+---+--
  |   |  
Your move (O). Choose one of these: [5, 6, 7, 8]
Enter your move (0-8): 6
You moved.
X | X | O
--+---+--
X | O |  
--+---+--
O |   |  

Game Over! Winner: O
